In [27]:
import time
import os
import openai
from freeplay import Freeplay, RecordPayload, CallInfo, ResponseInfo, SessionInfo, CallInfo
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

openai.api_key = os.environ.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
FREEPLAY_API_KEY = os.environ.get("FREEPLAY_API_KEY")

# Grabbing a row of CSV/Clinical trials that has references
df=pd.read_csv('ctg-studies-tiny.csv')
# summaries_df=df[['id', 'summary']].copy()

# summaries_df.head()

# df = pd.read_csv(input_datapath, index_col=0)
# df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Study ID: " + df.id.str.strip() + "; Summary: " + df.summary.str.strip() + "; Description: " + df.description.str.strip()
)

df.head()
# print(df.iloc[0].references)
# encoding = tiktoken.get_encoding(embedding_encoding)

# df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
# df = df[df.n_tokens <= max_tokens].tail(top_n)
# len(df)
# print(df.count())

,Unnamed: 0,id,completionDate,completionDateType,summary,description,conditions,studyType,eligibilityCriteria,healthyVolunteers,sex,minimumAge,maximumAge,references,combined
11,268564,NCT03442803,2018-12-31,ESTIMATED,There exists continued controversy over the us...,Procedural sedation and analgesia (PSA) has lo...,['Shoulder Dislocation'],INTERVENTIONAL,Inclusion Criteria:\n\n* Aged 18-65 years\n* C...,True,ALL,18 Years,65 Years,"[{'pmid': '33298604', 'type': 'DERIVED', 'cita...",Study ID: NCT03442803; Summary: There exists c...
14,375682,NCT02751125,2020-03-01,ACTUAL,This pilot study is aimed to reconstruct atrop...,Eleven patients presenting with the need of an...,['Bone Atrophy'],INTERVENTIONAL,Inclusion Criteria:Patients presenting with an...,False,ALL,18 Years,80 Years,"[{'pmid': '30092840', 'type': 'RESULT', 'citat...",Study ID: NCT02751125; Summary: This pilot stu...
26,25930,NCT03128723,2017-06-08,ACTUAL,The study will look to evaluate the tolerance ...,Acne and sensitive skin can be closely connect...,['Acne Vulgaris'],INTERVENTIONAL,Inclusion Criteria:\n\n* In good general healt...,False,ALL,12 Years,40 Years,"[{'pmid': '9744910', 'type': 'BACKGROUND', 'ci...",Study ID: NCT03128723; Summary: The study will...
36,50196,NCT01567969,2016-02,ACTUAL,The Family Health and Development Project (FHD...,The Family Health and Development Project (FHD...,['Serious Emotional Disturbance'],INTERVENTIONAL,Inclusion Criteria:\n\n* child has a Diagnosti...,False,ALL,6 Years,16 Years,"[{'type': 'BACKGROUND', 'citation': 'Woolston ...",Study ID: NCT01567969; Summary: The Family Hea...
39,391862,NCT05088096,2022-04-05,ACTUAL,A reliability and validity study for the cross...,Upper limb functional disorders are quite comm...,"['Upper Limb Nerve Lesion', 'Reliability', 'Va...",OBSERVATIONAL,Inclusion Criteria:\n\n* Both male and female ...,False,ALL,18 Years,40 Years,"[{'pmid': '35858863', 'type': 'DERIVED', 'cita...",Study ID: NCT05088096; Summary: A reliability ...


In [29]:
# data cleaning to drop any trials where 
# text splitting => embedding
# what's the best way to split?
# how to grab text from reference
# once text is chunked, pass chunks to embedding model
# store embeddings somewhere... for one document could probably keep in memory
# preform some kind of comparison
# put them in a db and query as part of 

In [31]:
# from langchain_community.document_loaders.csv_loader import CSVLoader
# from langchain_community.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import CSVLoader
# from openai.embeddings_utils import get_embedding

# def get_embedding(text, model="text-embedding-3-small"):
#    text = text.replace("\n", " ")
#    return client.embeddings.create(input = [text], model=model).data[0].embedding

# df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
# df.to_csv('output/embedded_1k_reviews.csv', index=False)


loader = CSVLoader('ctg-studies-tiny.csv')
data = loader.load()

# embeddings_model = OpenAIEmbeddings(
#     openai_api_key=OPEN_AI_KEY,
#     model="text-embedding-3-large"
#     # With the `text-embedding-3` class
#     # of models, you can specify the size
#     # of the embeddings you want returned.
#     # dimensions=1024
# )

# embeddings = embeddings_model.embed_documents(data)

# encoding = tiktoken.get_encoding("cl100k_base")
# df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
# df = df[df.n_tokens <= 8000]
# df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))
# df.head()
# db = DocArrayInMemorySearch.from_documents(data, embeddings_model)

# retriver = db.as_retriever()
# chain = RetrievalQA.from_chain_type(llm = llm, retriever = retriver, verbose=False, chain_type = "stuff")
# chain.run("your query")
# embeddings = embeddings_model.embed_documents(
#     [
#         "Hi there!",
#         "Oh, hello!"
#     ]
# )


In [ ]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
    text = df["combined"].replace("\n", " ")
    return client.embeddings.create(input = text, model=model).data[0].embedding

df['embeddings'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
embeddings_dict = dict(zip(df.id,df.embeddings))
# df.to_csv('output/embedded_1k_reviews.csv', index=False)
# df['embeddings'].head(10)
# print(embeddings_dict)

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

# conditions = ['Congenital heart disease']
condition_embedding = get_embedding('Congenital heart disease')

relevant_studies = []
for study, embedding in embeddings_dict.items():
    similarity = cosine_similarity([condition_embedding], [embedding])[0][0]
    relevant_studies.append((study, similarity))

relevant_studies.sort(key=lambda x: x[1], reverse=True)
# potential_conditions = [condition for condition, _ in relevant_conditions]

print("Potential conditions:", relevant_studies)

Potential conditions: [('NCT01567969', 1.0000000000000002), ('NCT03032419', 1.0000000000000002), ('NCT00078832', 1.0000000000000002), ('NCT01991795', 1.0000000000000002), ('NCT05657522', 1.0000000000000002), ('NCT02360956', 1.0000000000000002), ('NCT03928327', 1.0000000000000002), ('NCT03393104', 1.0000000000000002), ('NCT01027052', 1.0000000000000002), ('NCT01315548', 1.0000000000000002), ('NCT00770510', 1.0000000000000002), ('NCT05377853', 1.0000000000000002), ('NCT04692168', 1.0000000000000002), ('NCT02005653', 1.0000000000000002), ('NCT04032574', 1.0000000000000002), ('NCT00339326', 1.0000000000000002), ('NCT03545399', 1.0000000000000002), ('NCT00875654', 1.0000000000000002), ('NCT03356483', 1.0000000000000002), ('NCT05479552', 1.0000000000000002), ('NCT02210754', 1.0000000000000002), ('NCT02727348', 1.0000000000000002), ('NCT05708066', 0.9999998417053211), ('NCT05304247', 0.9999998379674426), ('NCT02722759', 0.9999998332887872), ('NCT02751125', 0.9999998306592175), ('NCT01598389',

In [59]:
study_id = "NCT03032419"
study_data = embeddings_dict[study_id]

# print("condition_embedding:", condition_embedding)
# print("study_data:", study_data)
similarity = cosine_similarity([condition_embedding], [study_data])[0][0]

print(similarity)

study_data: [9.408949699718505e-05, 0.0029640542343258858, 0.029289470985531807, 0.032713260501623154, -0.02248200960457325, 0.04477677866816521, 0.0005140703287906945, 0.04603395238518715, 0.004095844924449921, -0.03153633326292038, 0.02982443757355213, -0.02961045131087303, -0.02607966586947441, 0.01683810167014599, 0.05218607932329178, -0.06227021664381027, -0.01733294688165188, -0.007429360877722502, -0.014203387312591076, 0.001905821729451418, 0.030305908992886543, 0.032124798744916916, 0.007790463976562023, -0.01523319911211729, -0.0031613234896212816, -0.06050482392311096, 0.020663119852542877, -0.023739183321595192, -0.02955695427954197, 0.013581487350165844, 0.032258540391922, -0.028807999566197395, -0.030145417898893356, -0.038785144686698914, 0.009495672769844532, 0.019740300253033638, 0.004951792769134045, 0.03557533770799637, -0.037875697016716, -0.009809966199100018, 0.004838112276047468, -0.05133013054728508, -0.024795742705464363, 0.00047185804578475654, -0.058953419327